In [228]:
import cv2
import os
import numpy as np
import insightface
from PIL import Image
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

app = FaceAnalysis(name = 'buffalo_sc', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
#detect하고 square patches are cropped from the origina limage and resized to 640*640
app.prepare(ctx_id=0, det_size=(640, 640))


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '0', 'cudnn_conv1d_pad_to_nc1d': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\ParkMinSu/.insightface\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNex

In [229]:
def get_embed(path): # Input : rgb 이미지의 경로
                     # Output : 이미지에 있는 얼굴들의 임베딩
    img = np.asarray(Image.open(path))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # 이미지를 BGR 형식으로 바꿔 주어야 한다.
    faces = app.get(img)
    embeddings = []
    for tmp in faces:
        embeddings.append(tmp["embedding"])
    
    return embeddings

In [230]:
def similarity(embed_1, embed_2):
    n1 = np.linalg.norm(embed_1)
    n2 = np.linalg.norm(embed_2)
    return np.dot(embed_1, embed_2)/(n1*n2)

In [231]:
def dist(embed_1, embed_2):

    return np.linalg.norm(embed_1 - embed_2)

In [232]:
def embed_list(dir_path):
  embed_vectors=[]
  for (root, directories, files) in os.walk(dir_path):
    files = sorted(files)
    print(files)
    for file in files:
      file_path = os.path.join(root, file)
      print(file_path)
      embed_vectors.append(get_embed(file_path))
  return embed_vectors

In [233]:
def target_in_image(img_embeddings, target_embedding, threshold = 0.4): # Input : rgb 이미지의 경로, 사진 속에 있는지 확인하고 싶은 사람(타겟)의 임베딩, threshold
                                                              # Output : 이미지에 해당 사람(타겟)이 있는지 true/false
                                                              # 적절한 threshold 값을 선택하여 성능 향상을 노려볼 수 있을 것 같다.
    for embedding in img_embeddings:
        if(similarity(embedding, target_embedding[0]) > threshold):
            return 1 # target exist in image
    return 0 # target is not found in image

In [234]:

img_embed_list = embed_list('images/')
target_list = embed_list('targets/')
answer_str = ""
for target in target_list:
  for img in img_embed_list:
      a = target_in_image(img, target)
      answer_str+=str(a)
      print(a, end='')
  print()

print()
print(answer_str)


['0.png', '1.jfif', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg']
images/0.png
images/1.jfif
images/2.jpg
images/3.jpg
images/4.jpg
images/5.jpg
images/6.jpg
images/7.jpg
images/8.jpg
['0.jpg', '1.jpg', '2.jpg']
targets/0.jpg
targets/1.jpg
targets/2.jpg
010000000
000111110
110000000

010000000000111110110000000


In [235]:
def classified_image(image_list, target_list):
    answer_str = ""
    for target in target_list:
        target_embed = get_embed(target)
        for img in image_list:
            answer_str+str(target_in_image(img, target_embed))
    return answer_list

In [236]:
import os

dir_path = "sample_data/"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)